# 🔮 Predictive Modeling - Customer Analytics

This notebook focuses on building and training predictive models for customer analytics.

## Objectives:
- Build churn prediction models
- Develop Customer Lifetime Value (CLV) prediction
- Create recommendation systems
- Implement feature engineering for ML models
- Optimize model hyperparameters
- Validate model performance


In [7]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning imports
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

import warnings
warnings.filterwarnings('ignore')

# Import custom modules
import sys
sys.path.append('../src')
from components.predictive_modeling import PredictiveModeling
from utils.evaluation import calculate_model_metrics 

print("🤖 Predictive modeling libraries loaded!")

# Set random seed for reproducibility
np.random.seed(42)
print("🎯 Random seed set for reproducible results")

🤖 Predictive modeling libraries loaded!
🎯 Random seed set for reproducible results


In [8]:
# Load data
try:
    df = pd.read_csv('../data/processed/cleaned_data.csv')
    print(f"✅ Cleaned dataset loaded: {df.shape}")
except FileNotFoundError:
    print("📊 Loading raw data for processing...")
    try:
        df = pd.read_csv('../data/raw/customer_shopping_data.csv')
    except FileNotFoundError:
        from src.utils.common import load_sample_data
        df = load_sample_data(n_customers=2000)
    print(f"Dataset shape: {df.shape}")

# Load segmentation results if available
try:
    segmentation_df = pd.read_csv('../reports/analysis/03_customer_segmentation_results.csv')
    print(f"✅ Segmentation results loaded: {segmentation_df.shape}")
except FileNotFoundError:
    print("⚠️ Segmentation results not found. Will create synthetic segments.")
    segmentation_df = None

print(f"\n📋 Dataset Overview:")
print(f"Records: {len(df):,}")
if 'Customer ID' in df.columns:
    print(f"Unique Customers: {df['Customer ID'].nunique():,}")
print(f"Features: {df.shape[1]}")
print(f"Columns: {list(df.columns)}")

📊 Loading raw data for processing...
Dataset shape: (3900, 19)
✅ Segmentation results loaded: (3900, 11)

📋 Dataset Overview:
Records: 3,900
Unique Customers: 3,900
Features: 19
Columns: ['Customer ID', 'Age', 'Gender', 'Item Purchased', 'Category', 'Purchase Amount (USD)', 'Location', 'Size', 'Color', 'Season', 'Review Rating', 'Subscription Status', 'Payment Method', 'Shipping Type', 'Discount Applied', 'Promo Code Used', 'Previous Purchases', 'Preferred Payment Method', 'Frequency of Purchases']


## 1. Feature Engineering for Predictive Modeling

In [9]:
# Advanced feature engineering
def create_predictive_features(df):
    """Create features for predictive modeling"""
    
    features_df = df.copy()
    
    print("🔧 Creating predictive features...")
    
    # Date-based features
    if 'Purchase Date' in df.columns:
        features_df['Purchase Date'] = pd.to_datetime(features_df['Purchase Date'])
        features_df['Purchase_Year'] = features_df['Purchase Date'].dt.year
        features_df['Purchase_Month'] = features_df['Purchase Date'].dt.month
        features_df['Purchase_Quarter'] = features_df['Purchase Date'].dt.quarter
        features_df['Purchase_DayOfWeek'] = features_df['Purchase Date'].dt.dayofweek
        features_df['Purchase_Weekend'] = (features_df['Purchase_DayOfWeek'] >= 5).astype(int)
        
        # Days since reference date
        reference_date = features_df['Purchase Date'].max()
        features_df['Days_Since_Purchase'] = (reference_date - features_df['Purchase Date']).dt.days
        
        print("  ✅ Date features created")
    
    # Customer-level aggregated features
    if 'Customer ID' in df.columns:
        customer_features = df.groupby('Customer ID').agg({
            'Purchase Amount (USD)': ['sum', 'mean', 'std', 'count', 'min', 'max'] if 'Purchase Amount (USD)' in df.columns else None,
            'Review Rating': ['mean', 'std', 'count'] if 'Review Rating' in df.columns else None,
            'Category': ['nunique', 'count'] if 'Category' in df.columns else None,
            'Age': 'first' if 'Age' in df.columns else None
        }).fillna(0)
        
        # Flatten column names
        customer_features.columns = ['_'.join(col).strip() if col[1] else col[0] for col in customer_features.columns.values]
        
        # Additional derived features
        if 'Purchase Amount (USD)' in df.columns:
            customer_features['Purchase_Amount_CV'] = customer_features['Purchase Amount (USD)_std'] / (customer_features['Purchase Amount (USD)_mean'] + 1e-10)
            customer_features['Revenue_Per_Transaction'] = customer_features['Purchase Amount (USD)_sum'] / (customer_features['Purchase Amount (USD)_count'] + 1e-10)
            customer_features['High_Value_Transactions'] = (customer_features['Purchase Amount (USD)_max'] > customer_features['Purchase Amount (USD)_mean'] * 2).astype(int)
        
        # Merge back to main dataframe
        features_df = features_df.merge(customer_features, left_on='Customer ID', right_index=True, how='left')
        
        print("  ✅ Customer aggregated features created")
    
    # Categorical encoding
    categorical_columns = features_df.select_dtypes(include=['object']).columns
    categorical_columns = [col for col in categorical_columns if col not in ['Customer ID', 'Purchase Date']]
    
    for col in categorical_columns:
        if features_df[col].nunique() <= 10:  # One-hot encode if few categories
            encoded = pd.get_dummies(features_df[col], prefix=col)
            features_df = pd.concat([features_df, encoded], axis=1)
        else:  # Label encode if many categories
            le = LabelEncoder()
            features_df[f'{col}_encoded'] = le.fit_transform(features_df[col].astype(str))
    
    print("  ✅ Categorical encoding completed")
    
    # Interaction features
    if 'Age' in features_df.columns and 'Purchase Amount (USD)' in features_df.columns:
        features_df['Age_PurchaseAmount_Interaction'] = features_df['Age'] * features_df['Purchase Amount (USD)']
        
    if 'Review Rating' in features_df.columns and 'Purchase Amount (USD)' in features_df.columns:
        features_df['Rating_PurchaseAmount_Interaction'] = features_df['Review Rating'] * features_df['Purchase Amount (USD)']
    
    print("  ✅ Interaction features created")
    
    print(f"\n📊 Feature engineering completed:")
    print(f"  Original features: {df.shape[1]}")
    print(f"  New features: {features_df.shape[1]}")
    print(f"  Added features: {features_df.shape[1] - df.shape[1]}")
    
    return features_df

# Create features
model_features = create_predictive_features(df)
print(f"\n✅ Features created successfully: {model_features.shape}")

🔧 Creating predictive features...
  ✅ Customer aggregated features created
  ✅ Categorical encoding completed
  ✅ Interaction features created

📊 Feature engineering completed:
  Original features: 19
  New features: 84
  Added features: 65

✅ Features created successfully: (3900, 84)


## 2. Churn Prediction Model

In [10]:
# Create churn labels (synthetic for demonstration)
def create_churn_labels(df):
    """Create churn labels based on business logic"""
    
    print("🎯 Creating churn labels...")
    
    # Use recency and other factors to determine churn probability
    churn_factors = pd.DataFrame()
    
    if 'Days_Since_Purchase' in df.columns:
        # Customers who haven't purchased in 90+ days are more likely to churn
        churn_factors['recency_risk'] = (df['Days_Since_Purchase'] > 90).astype(int) * 0.4
    else:
        churn_factors['recency_risk'] = np.random.random(len(df)) * 0.4
    
    if 'Review Rating' in df.columns:
        # Customers with low ratings are more likely to churn
        churn_factors['satisfaction_risk'] = ((5 - df['Review Rating']) / 5) * 0.3
    else:
        churn_factors['satisfaction_risk'] = np.random.random(len(df)) * 0.3
    
    if 'Purchase Amount (USD)_count' in df.columns:
        # Customers with very few purchases are more likely to churn
        churn_factors['frequency_risk'] = (1 / (df['Purchase Amount (USD)_count'] + 1)) * 0.3
    else:
        churn_factors['frequency_risk'] = np.random.random(len(df)) * 0.3
    
    # Calculate overall churn probability
    churn_probability = churn_factors.sum(axis=1)
    churn_probability = np.clip(churn_probability, 0, 1)
    
    # Generate binary churn labels
    np.random.seed(42)
    churn_labels = np.random.binomial(1, churn_probability)
    
    churn_rate = churn_labels.mean()
    print(f"  ✅ Churn labels created: {churn_rate:.1%} churn rate")
    
    return churn_labels, churn_probability

# Create churn labels
churn_labels, churn_probability = create_churn_labels(model_features)
model_features['Churn'] = churn_labels
model_features['Churn_Probability'] = churn_probability

print(f"\n📊 Churn distribution:")
churn_dist = pd.Series(churn_labels).value_counts()
for label, count in churn_dist.items():
    percentage = count / len(churn_labels) * 100
    label_name = 'Churned' if label == 1 else 'Active'
    print(f"  {label_name}: {count:,} ({percentage:.1f}%)")

🎯 Creating churn labels...
  ✅ Churn labels created: 14.4% churn rate

📊 Churn distribution:
  Active: 3,338 (85.6%)
  Churned: 562 (14.4%)


In [11]:
# Prepare data for churn prediction
def prepare_churn_modeling_data(df):
    """Prepare features for churn prediction model"""
    
    # Select relevant features for modeling
    feature_columns = []
    
    # Numeric features
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    exclude_cols = ['Churn', 'Churn_Probability', 'Customer ID']
    feature_columns.extend([col for col in numeric_cols if col not in exclude_cols])
    
    # Select features that exist in the dataframe
    available_features = [col for col in feature_columns if col in df.columns]
    
    if len(available_features) == 0:
        print("⚠️ No suitable features found. Using basic features.")
        # Use basic available numeric columns
        available_features = [col for col in df.select_dtypes(include=[np.number]).columns 
                            if col not in exclude_cols][:10]  # Take first 10 numeric columns
    
    X = df[available_features].fillna(0)
    y = df['Churn']
    
    print(f"📊 Churn modeling data prepared:")
    print(f"  Features: {X.shape[1]}")
    print(f"  Samples: {X.shape[0]}")
    print(f"  Selected features: {available_features[:5]}...")  # Show first 5
    
    return X, y, available_features

# Prepare churn modeling data
X_churn, y_churn, churn_features = prepare_churn_modeling_data(model_features)

# Split data
X_churn_train, X_churn_test, y_churn_train, y_churn_test = train_test_split(
    X_churn, y_churn, test_size=0.2, random_state=42, stratify=y_churn
)

print(f"\n📈 Data split completed:")
print(f"  Training set: {X_churn_train.shape[0]} samples")
print(f"  Test set: {X_churn_test.shape[0]} samples")
print(f"  Training churn rate: {y_churn_train.mean():.1%}")
print(f"  Test churn rate: {y_churn_test.mean():.1%}")

📊 Churn modeling data prepared:
  Features: 24
  Samples: 3900
  Selected features: ['Age', 'Purchase Amount (USD)', 'Review Rating', 'Previous Purchases', 'Purchase Amount (USD)_sum']...

📈 Data split completed:
  Training set: 3120 samples
  Test set: 780 samples
  Training churn rate: 14.4%
  Test churn rate: 14.4%


In [12]:
# Build and train churn prediction models
def train_churn_models(X_train, y_train, X_test, y_test):
    """Train multiple churn prediction models"""
    
    models = {
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced'),
        'Logistic Regression': LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
        'Gradient Boosting': GradientBoostingClassifier(n_estimators=100, random_state=42)
    }
    
    results = {}
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print("🤖 Training churn prediction models...")
    
    for name, model in models.items():
        print(f"\n  Training {name}...")
        
        # Train model
        if name == 'Logistic Regression':
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
            y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        
        # Calculate metrics
        accuracy = (y_pred == y_test).mean()
        auc_score = roc_auc_score(y_test, y_pred_proba)
        
        # Cross-validation score
        cv_scores = cross_val_score(model, X_train_scaled if name == 'Logistic Regression' else X_train, 
                                  y_train, cv=5, scoring='roc_auc')
        
        results[name] = {
            'model': model,
            'accuracy': accuracy,
            'auc_score': auc_score,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'predictions': y_pred,
            'probabilities': y_pred_proba
        }
        
        print(f"    Accuracy: {accuracy:.3f}")
        print(f"    AUC Score: {auc_score:.3f}")
        print(f"    CV Score: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
    
    return results, scaler

# Train churn models
churn_model_results, churn_scaler = train_churn_models(X_churn_train, y_churn_train, X_churn_test, y_churn_test)

# Find best model
best_churn_model_name = max(churn_model_results.keys(), 
                           key=lambda x: churn_model_results[x]['auc_score'])
best_churn_model = churn_model_results[best_churn_model_name]

print(f"\n🏆 Best churn model: {best_churn_model_name}")
print(f"  AUC Score: {best_churn_model['auc_score']:.3f}")
print(f"  Accuracy: {best_churn_model['accuracy']:.3f}")

🤖 Training churn prediction models...

  Training Random Forest...
    Accuracy: 0.854
    AUC Score: 0.607
    CV Score: 0.609 ± 0.019

  Training Logistic Regression...
    Accuracy: 0.614
    AUC Score: 0.681
    CV Score: 0.676 ± 0.019

  Training Gradient Boosting...
    Accuracy: 0.855
    AUC Score: 0.642
    CV Score: 0.627 ± 0.030

🏆 Best churn model: Logistic Regression
  AUC Score: 0.681
  Accuracy: 0.614


In [13]:
# Visualize churn model results
def visualize_churn_results(results, y_test):
    """Visualize churn prediction results"""
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['Model Performance Comparison', 'ROC Curves', 
                       'Feature Importance (Best Model)', 'Churn Probability Distribution']
    )
    
    # Model performance comparison
    model_names = list(results.keys())
    accuracies = [results[name]['accuracy'] for name in model_names]
    auc_scores = [results[name]['auc_score'] for name in model_names]
    
    fig.add_trace(
        go.Bar(x=model_names, y=accuracies, name='Accuracy', offsetgroup=1),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Bar(x=model_names, y=auc_scores, name='AUC Score', offsetgroup=2),
        row=1, col=1
    )
    
    # ROC Curves
    for name in model_names:
        y_pred_proba = results[name]['probabilities']
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        
        fig.add_trace(
            go.Scatter(x=fpr, y=tpr, name=f'{name} (AUC={results[name]["auc_score"]:.3f})',
                      mode='lines'),
            row=1, col=2
        )
    
    # Add diagonal line for ROC
    fig.add_trace(
        go.Scatter(x=[0, 1], y=[0, 1], mode='lines', line=dict(dash='dash'),
                  name='Random', showlegend=False),
        row=1, col=2
    )
    
    # Feature importance (for tree-based models)
    best_model = results[best_churn_model_name]['model']
    if hasattr(best_model, 'feature_importances_'):
        feature_imp = pd.DataFrame({
            'feature': churn_features,
            'importance': best_model.feature_importances_
        }).sort_values('importance', ascending=True).tail(10)
        
        fig.add_trace(
            go.Bar(x=feature_imp['importance'], y=feature_imp['feature'], 
                   orientation='h', name='Importance'),
            row=2, col=1
        )
    
    # Churn probability distribution
    best_probabilities = results[best_churn_model_name]['probabilities']
    
    fig.add_trace(
        go.Histogram(x=best_probabilities[y_test == 0], name='Active Customers', 
                    opacity=0.7, nbinsx=20),
        row=2, col=2
    )
    
    fig.add_trace(
        go.Histogram(x=best_probabilities[y_test == 1], name='Churned Customers', 
                    opacity=0.7, nbinsx=20),
        row=2, col=2
    )
    
    fig.update_layout(height=800, title_text="Churn Prediction Model Analysis")
    fig.show()

# Visualize results
visualize_churn_results(churn_model_results, y_churn_test)

# Print detailed classification report for best model
print(f"\n📊 DETAILED CLASSIFICATION REPORT - {best_churn_model_name}:")
print("=" * 60)
best_predictions = churn_model_results[best_churn_model_name]['predictions']
print(classification_report(y_churn_test, best_predictions, 
                          target_names=['Active', 'Churned']))


📊 DETAILED CLASSIFICATION REPORT - Logistic Regression:
              precision    recall  f1-score   support

      Active       0.92      0.60      0.73       668
     Churned       0.22      0.67      0.33       112

    accuracy                           0.61       780
   macro avg       0.57      0.64      0.53       780
weighted avg       0.82      0.61      0.67       780



## 3. Customer Lifetime Value (CLV) Prediction

In [14]:
# Create CLV labels
def create_clv_labels(df):
    """Create CLV labels based on customer behavior"""
    
    print("💎 Creating CLV labels...")
    
    # Base CLV calculation
    clv_factors = pd.DataFrame()
    
    if 'Purchase Amount (USD)_sum' in df.columns:
        clv_factors['historical_value'] = df['Purchase Amount (USD)_sum']
    else:
        clv_factors['historical_value'] = np.random.exponential(300, len(df))
    
    if 'Purchase Amount (USD)_count' in df.columns:
        clv_factors['frequency_factor'] = df['Purchase Amount (USD)_count'] / 12  # Monthly frequency
    else:
        clv_factors['frequency_factor'] = np.random.poisson(2, len(df))
    
    if 'Purchase Amount (USD)_mean' in df.columns:
        clv_factors['avg_order_value'] = df['Purchase Amount (USD)_mean']
    else:
        clv_factors['avg_order_value'] = np.random.exponential(50, len(df))
    
    # Customer lifespan (in months) - estimate based on engagement
    if 'Review Rating' in df.columns:
        satisfaction_factor = df['Review Rating'] / 5  # Normalize to 0-1
        estimated_lifespan = 12 + satisfaction_factor * 24  # 12-36 months based on satisfaction
    else:
        estimated_lifespan = np.random.uniform(12, 36, len(df))
    
    # CLV = Average Order Value * Purchase Frequency * Customer Lifespan
    predicted_clv = (
        clv_factors['avg_order_value'] * 
        clv_factors['frequency_factor'] * 
        estimated_lifespan
    )
    
    # Add some noise and ensure positive values
    np.random.seed(42)
    noise = np.random.normal(0, predicted_clv.std() * 0.1, len(predicted_clv))
    predicted_clv = np.maximum(predicted_clv + noise, 10)  # Minimum CLV of $10
    
    print(f"  ✅ CLV labels created")
    print(f"  Mean CLV: ${predicted_clv.mean():.2f}")
    print(f"  Median CLV: ${predicted_clv.median():.2f}")
    print(f"  CLV Range: ${predicted_clv.min():.2f} - ${predicted_clv.max():.2f}")
    
    return predicted_clv

# Create CLV labels
clv_labels = create_clv_labels(model_features)
model_features['CLV'] = clv_labels

# CLV distribution analysis
print(f"\n📊 CLV Distribution Analysis:")
clv_quartiles = pd.qcut(clv_labels, 4, labels=['Low', 'Medium', 'High', 'Very High'])
clv_dist = clv_quartiles.value_counts()
for quartile, count in clv_dist.items():
    percentage = count / len(clv_labels) * 100
    avg_clv = clv_labels[clv_quartiles == quartile].mean()
    print(f"  {quartile} CLV: {count:,} customers ({percentage:.1f}%) - Avg: ${avg_clv:.2f}")

💎 Creating CLV labels...
  ✅ CLV labels created
  Mean CLV: $149.73
  Median CLV: $147.09
  CLV Range: $33.36 - $308.79

📊 CLV Distribution Analysis:
  Low CLV: 975 customers (25.0%) - Avg: $71.13
  Medium CLV: 975 customers (25.0%) - Avg: $120.90
  High CLV: 975 customers (25.0%) - Avg: $173.54
  Very High CLV: 975 customers (25.0%) - Avg: $233.35


In [15]:
# Prepare data for CLV prediction
def prepare_clv_modeling_data(df):
    """Prepare features for CLV prediction model"""
    
    # Select relevant features for CLV modeling
    feature_columns = []
    
    # Numeric features (exclude CLV itself)
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    exclude_cols = ['CLV', 'Churn', 'Churn_Probability', 'Customer ID']
    feature_columns.extend([col for col in numeric_cols if col not in exclude_cols])
    
    # Select features that exist in the dataframe
    available_features = [col for col in feature_columns if col in df.columns]
    
    if len(available_features) == 0:
        print("⚠️ No suitable features found. Using basic features.")
        available_features = [col for col in df.select_dtypes(include=[np.number]).columns 
                            if col not in exclude_cols][:10]
    
    X = df[available_features].fillna(0)
    y = df['CLV']
    
    print(f"📊 CLV modeling data prepared:")
    print(f"  Features: {X.shape[1]}")
    print(f"  Samples: {X.shape[0]}")
    print(f"  Selected features: {available_features[:5]}...")
    
    return X, y, available_features

# Prepare CLV modeling data
X_clv, y_clv, clv_features = prepare_clv_modeling_data(model_features)

# Split data
X_clv_train, X_clv_test, y_clv_train, y_clv_test = train_test_split(
    X_clv, y_clv, test_size=0.2, random_state=42
)

print(f"\n📈 CLV data split completed:")
print(f"  Training set: {X_clv_train.shape[0]} samples")
print(f"  Test set: {X_clv_test.shape[0]} samples")
print(f"  Training CLV range: ${y_clv_train.min():.2f} - ${y_clv_train.max():.2f}")
print(f"  Test CLV range: ${y_clv_test.min():.2f} - ${y_clv_test.max():.2f}")

📊 CLV modeling data prepared:
  Features: 24
  Samples: 3900
  Selected features: ['Age', 'Purchase Amount (USD)', 'Review Rating', 'Previous Purchases', 'Purchase Amount (USD)_sum']...

📈 CLV data split completed:
  Training set: 3120 samples
  Test set: 780 samples
  Training CLV range: $33.36 - $308.79
  Test CLV range: $36.56 - $299.40


In [17]:
from sklearn.ensemble import GradientBoostingRegressor


# Build and train CLV prediction models
def train_clv_models(X_train, y_train, X_test, y_test):
    """Train multiple CLV prediction models"""
    
    models = {
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'Linear Regression': LinearRegression(),
        'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42)
    }
    
    results = {}
    
    # Scale features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    print("💎 Training CLV prediction models...")
    
    for name, model in models.items():
        print(f"\n  Training {name}...")
        
        # Train model
        if name == 'Linear Regression':
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
        
        # Calculate metrics
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        rmse = np.sqrt(mse)
        
        # Cross-validation score
        cv_scores = cross_val_score(model, X_train_scaled if name == 'Linear Regression' else X_train, 
                                  y_train, cv=5, scoring='r2')
        
        results[name] = {
            'model': model,
            'mse': mse,
            'mae': mae,
            'rmse': rmse,
            'r2': r2,
            'cv_mean': cv_scores.mean(),
            'cv_std': cv_scores.std(),
            'predictions': y_pred
        }
        
        print(f"    R² Score: {r2:.3f}")
        print(f"    RMSE: ${rmse:.2f}")
        print(f"    MAE: ${mae:.2f}")
        print(f"    CV R² Score: {cv_scores.mean():.3f} ± {cv_scores.std():.3f}")
    
    return results, scaler

# Train CLV models
clv_model_results, clv_scaler = train_clv_models(X_clv_train, y_clv_train, X_clv_test, y_clv_test)

# Find best model
best_clv_model_name = max(clv_model_results.keys(), 
                         key=lambda x: clv_model_results[x]['r2'])
best_clv_model = clv_model_results[best_clv_model_name]

print(f"\n🏆 Best CLV model: {best_clv_model_name}")
print(f"  R² Score: {best_clv_model['r2']:.3f}")
print(f"  RMSE: ${best_clv_model['rmse']:.2f}")
print(f"  MAE: ${best_clv_model['mae']:.2f}")

💎 Training CLV prediction models...

  Training Random Forest...
    R² Score: 0.989
    RMSE: $6.74
    MAE: $5.40
    CV R² Score: 0.988 ± 0.001

  Training Linear Regression...
    R² Score: 0.991
    RMSE: $6.14
    MAE: $4.84
    CV R² Score: 0.990 ± 0.001

  Training Gradient Boosting...
    R² Score: 0.990
    RMSE: $6.45
    MAE: $5.11
    CV R² Score: 0.989 ± 0.001

🏆 Best CLV model: Linear Regression
  R² Score: 0.991
  RMSE: $6.14
  MAE: $4.84


In [18]:
# Visualize CLV model results
def visualize_clv_results(results, y_test):
    """Visualize CLV prediction results"""
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['Model Performance Comparison', 'Actual vs Predicted (Best Model)', 
                       'Feature Importance (Best Model)', 'Prediction Error Distribution']
    )
    
    # Model performance comparison
    model_names = list(results.keys())
    r2_scores = [results[name]['r2'] for name in model_names]
    rmse_scores = [results[name]['rmse'] for name in model_names]
    
    fig.add_trace(
        go.Bar(x=model_names, y=r2_scores, name='R² Score'),
        row=1, col=1
    )
    
    # Actual vs Predicted scatter plot
    best_predictions = results[best_clv_model_name]['predictions']
    
    fig.add_trace(
        go.Scatter(x=y_test, y=best_predictions, mode='markers', name='Predictions',
                  opacity=0.6),
        row=1, col=2
    )
    
    # Perfect prediction line
    min_val = min(y_test.min(), best_predictions.min())
    max_val = max(y_test.max(), best_predictions.max())
    fig.add_trace(
        go.Scatter(x=[min_val, max_val], y=[min_val, max_val], mode='lines',
                  line=dict(dash='dash', color='red'), name='Perfect Prediction',
                  showlegend=False),
        row=1, col=2
    )
    
    # Feature importance
    best_model = results[best_clv_model_name]['model']
    if hasattr(best_model, 'feature_importances_'):
        feature_imp = pd.DataFrame({
            'feature': clv_features,
            'importance': best_model.feature_importances_
        }).sort_values('importance', ascending=True).tail(10)
        
        fig.add_trace(
            go.Bar(x=feature_imp['importance'], y=feature_imp['feature'], 
                   orientation='h', name='Importance'),
            row=2, col=1
        )
    
    # Prediction error distribution
    errors = best_predictions - y_test
    
    fig.add_trace(
        go.Histogram(x=errors, name='Prediction Errors', nbinsx=30),
        row=2, col=2
    )
    
    fig.update_layout(height=800, title_text="CLV Prediction Model Analysis")
    fig.show()

# Visualize results
visualize_clv_results(clv_model_results, y_clv_test)

# Print detailed metrics for all models
print(f"\n📊 DETAILED CLV MODEL COMPARISON:")
print("=" * 70)
comparison_df = pd.DataFrame({
    'Model': list(clv_model_results.keys()),
    'R² Score': [clv_model_results[name]['r2'] for name in clv_model_results.keys()],
    'RMSE': [clv_model_results[name]['rmse'] for name in clv_model_results.keys()],
    'MAE': [clv_model_results[name]['mae'] for name in clv_model_results.keys()],
    'CV R² Mean': [clv_model_results[name]['cv_mean'] for name in clv_model_results.keys()],
    'CV R² Std': [clv_model_results[name]['cv_std'] for name in clv_model_results.keys()]
})
display(comparison_df.round(3))


📊 DETAILED CLV MODEL COMPARISON:


,Model,R² Score,RMSE,MAE,CV R² Mean,CV R² Std
0,Random Forest,0.989,6.743,5.396,0.988,0.001
1,Linear Regression,0.991,6.139,4.839,0.990,0.001
2,Gradient Boosting,0.990,6.447,5.107,0.989,0.001


## 4. Model Optimization and Hyperparameter Tuning

In [19]:
# Hyperparameter tuning for best models
def optimize_churn_model(X_train, y_train):
    """Optimize hyperparameters for churn prediction model"""
    
    print("🎯 Optimizing churn prediction model...")
    
    # Define parameter grid for Random Forest
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 15, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    # Grid search with cross-validation
    rf_model = RandomForestClassifier(random_state=42, class_weight='balanced')
    
    grid_search = GridSearchCV(
        rf_model, param_grid, cv=5, scoring='roc_auc', 
        n_jobs=-1, verbose=1
    )
    
    grid_search.fit(X_train, y_train)
    
    print(f"  ✅ Best parameters: {grid_search.best_params_}")
    print(f"  ✅ Best CV score: {grid_search.best_score_:.3f}")
    
    return grid_search.best_estimator_, grid_search.best_params_

def optimize_clv_model(X_train, y_train):
    """Optimize hyperparameters for CLV prediction model"""
    
    print("\n💎 Optimizing CLV prediction model...")
    
    # Define parameter grid for Random Forest Regressor
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 15, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    # Grid search with cross-validation
    rf_model = RandomForestRegressor(random_state=42)
    
    grid_search = GridSearchCV(
        rf_model, param_grid, cv=5, scoring='r2', 
        n_jobs=-1, verbose=1
    )
    
    grid_search.fit(X_train, y_train)
    
    print(f"  ✅ Best parameters: {grid_search.best_params_}")
    print(f"  ✅ Best CV score: {grid_search.best_score_:.3f}")
    
    return grid_search.best_estimator_, grid_search.best_params_

# Optimize models (using smaller parameter grids for faster execution)
try:
    optimized_churn_model, churn_best_params = optimize_churn_model(X_churn_train, y_churn_train)
    optimized_clv_model, clv_best_params = optimize_clv_model(X_clv_train, y_clv_train)
except Exception as e:
    print(f"⚠️ Optimization failed: {e}")
    print("Using default models instead...")
    optimized_churn_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')
    optimized_clv_model = RandomForestRegressor(n_estimators=100, random_state=42)
    
    # Train with default parameters
    optimized_churn_model.fit(X_churn_train, y_churn_train)
    optimized_clv_model.fit(X_clv_train, y_clv_train)
    
    churn_best_params = {'n_estimators': 100, 'random_state': 42}
    clv_best_params = {'n_estimators': 100, 'random_state': 42}

print("\n✅ Model optimization completed!")

🎯 Optimizing churn prediction model...
Fitting 5 folds for each of 108 candidates, totalling 540 fits
  ✅ Best parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}
  ✅ Best CV score: 0.654

💎 Optimizing CLV prediction model...
Fitting 5 folds for each of 108 candidates, totalling 540 fits
  ✅ Best parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}
  ✅ Best CV score: 0.989

✅ Model optimization completed!


In [20]:
# Evaluate optimized models
def evaluate_optimized_models():
    """Evaluate the performance of optimized models"""
    
    print("📊 EVALUATING OPTIMIZED MODELS")
    print("=" * 50)
    
    # Churn model evaluation
    churn_pred = optimized_churn_model.predict(X_churn_test)
    churn_pred_proba = optimized_churn_model.predict_proba(X_churn_test)[:, 1]
    
    churn_accuracy = (churn_pred == y_churn_test).mean()
    churn_auc = roc_auc_score(y_churn_test, churn_pred_proba)
    
    print(f"\n🎯 OPTIMIZED CHURN MODEL:")
    print(f"  Accuracy: {churn_accuracy:.3f}")
    print(f"  AUC Score: {churn_auc:.3f}")
    print(f"  Best Parameters: {churn_best_params}")
    
    # CLV model evaluation
    clv_pred = optimized_clv_model.predict(X_clv_test)
    
    clv_r2 = r2_score(y_clv_test, clv_pred)
    clv_rmse = np.sqrt(mean_squared_error(y_clv_test, clv_pred))
    clv_mae = mean_absolute_error(y_clv_test, clv_pred)
    
    print(f"\n💎 OPTIMIZED CLV MODEL:")
    print(f"  R² Score: {clv_r2:.3f}")
    print(f"  RMSE: ${clv_rmse:.2f}")
    print(f"  MAE: ${clv_mae:.2f}")
    print(f"  Best Parameters: {clv_best_params}")
    
    return {
        'churn': {
            'model': optimized_churn_model,
            'accuracy': churn_accuracy,
            'auc': churn_auc,
            'predictions': churn_pred,
            'probabilities': churn_pred_proba
        },
        'clv': {
            'model': optimized_clv_model,
            'r2': clv_r2,
            'rmse': clv_rmse,
            'mae': clv_mae,
            'predictions': clv_pred
        }
    }

# Evaluate optimized models
final_model_results = evaluate_optimized_models()

📊 EVALUATING OPTIMIZED MODELS

🎯 OPTIMIZED CHURN MODEL:
  Accuracy: 0.605
  AUC Score: 0.673
  Best Parameters: {'max_depth': 5, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 50}

💎 OPTIMIZED CLV MODEL:
  R² Score: 0.989
  RMSE: $6.58
  MAE: $5.24
  Best Parameters: {'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 200}


In [21]:
# Save model results and predictions
try:
    # Create predictions dataframe
    predictions_df = pd.DataFrame({
        'Actual_Churn': y_churn_test.values,
        'Predicted_Churn': final_model_results['churn']['predictions'],
        'Churn_Probability': final_model_results['churn']['probabilities'],
        'Actual_CLV': y_clv_test.values,
        'Predicted_CLV': final_model_results['clv']['predictions']
    })
    
    # Save predictions
    predictions_df.to_csv('../reports/analysis/04_model_predictions.csv', index=False)
    print("💾 Model predictions saved to ../reports/analysis/04_model_predictions.csv")
    
    # Save model performance summary
    model_summary = pd.DataFrame({
        'Model': ['Churn Prediction', 'CLV Prediction'],
        'Algorithm': ['Random Forest Classifier', 'Random Forest Regressor'],
        'Primary_Metric': [final_model_results['churn']['auc'], final_model_results['clv']['r2']],
        'Metric_Name': ['AUC Score', 'R² Score'],
        'Secondary_Metric': [final_model_results['churn']['accuracy'], final_model_results['clv']['rmse']],
        'Secondary_Metric_Name': ['Accuracy', 'RMSE']
    })
    
    model_summary.to_csv('../reports/analysis/04_model_performance_summary.csv', index=False)
    print("💾 Model performance summary saved to ../reports/analysis/04_model_performance_summary.csv")
    
    print("\n✅ Predictive modeling completed successfully!")
    print("🚀 Ready for next notebook: 05_model_evaluation.ipynb")
    
except Exception as e:
    print(f"⚠️ Could not save results: {e}")
    print("📊 Analysis completed - results available in notebook")

# Final summary
print("\n" + "=" * 60)
print("📋 PREDICTIVE MODELING SUMMARY")
print("=" * 60)
print(f"✅ Feature Engineering: {model_features.shape[1]} features created")
print(f"✅ Churn Prediction: AUC = {final_model_results['churn']['auc']:.3f}, Accuracy = {final_model_results['churn']['accuracy']:.3f}")
print(f"✅ CLV Prediction: R² = {final_model_results['clv']['r2']:.3f}, RMSE = ${final_model_results['clv']['rmse']:.2f}")
print(f"✅ Models Optimized: Hyperparameter tuning completed")
print(f"✅ Results Saved: Predictions and performance metrics exported")
print("\n🎯 Key Recommendation: Implement churn prevention for high-risk customers and focus on high CLV segments")

💾 Model predictions saved to ../reports/analysis/04_model_predictions.csv
💾 Model performance summary saved to ../reports/analysis/04_model_performance_summary.csv

✅ Predictive modeling completed successfully!
🚀 Ready for next notebook: 05_model_evaluation.ipynb

📋 PREDICTIVE MODELING SUMMARY
✅ Feature Engineering: 87 features created
✅ Churn Prediction: AUC = 0.673, Accuracy = 0.605
✅ CLV Prediction: R² = 0.989, RMSE = $6.58
✅ Models Optimized: Hyperparameter tuning completed
✅ Results Saved: Predictions and performance metrics exported

🎯 Key Recommendation: Implement churn prevention for high-risk customers and focus on high CLV segments
